# Insiders Analysis

Transaction Codes

P: Open market or private purchase of non-derivative or derivative security

M: Exercise or conversion of derivative security exempted puruant to Rule 16b-3. Indicates
   that an insider has exercised options, typically converting them into shares. Can be seen as a form
   of indirect buying, though its a bit more nuanced because the insider may immediately sell the shares 
   they've acquired

S: Indicates an open market or private sale of a non derivative or derivative security.

In [10]:

import pandas as pd
import nasdaqdatalink

import numpy as np
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
NASDAQ_KEY = "zxLNKQydu_qNXQ2tZ7vz"

nasdaqdatalink.ApiConfig.api_key = NASDAQ_KEY


## Requesting Insider Data 

In [15]:
insiders = nasdaqdatalink.get_table('SHARADAR/SF2', ticker='CHTR')
insiders_df = pd.DataFrame(insiders, columns=["ticker", 'filingdate', 'transactiondate', 'formtype', 'issuername', 'ownername', 'officertitle', 'istenpercentowner', 'transactioncode', 'transactionpricepershare', 'transactionvalue', 'securitytitle'])
insiders_df = insiders_df.sort_values(by="filingdate", ascending=False)
insiders_df

,ticker,filingdate,transactiondate,formtype,issuername,ownername,officertitle,istenpercentowner,transactioncode,transactionpricepershare,transactionvalue,securitytitle
None,,,,,,,,,,,,
568,CHTR,2024-01-05,2024-01-04,4,CHARTER COMMUNICATIONS IN,NEWHOUSE MICHAEL A,None,N,D,NaN,NaN,Class B Common Units of Charter Communications...
2536,CHTR,2024-01-05,2024-01-04,4,CHARTER COMMUNICATIONS IN,ADVANCENEWHOUSE PARTNERSHIP,None,Y,D,NaN,NaN,Class B Common Units of Charter Communications...
1316,CHTR,2023-12-15,2023-12-15,4,CHARTER COMMUNICATIONS IN,LIBERTY BROADBAND CORP,None,Y,D,407.43,134134512.0,Class A Common Stock
2537,CHTR,2023-12-07,2023-12-06,4,CHARTER COMMUNICATIONS IN,ADVANCENEWHOUSE PARTNERSHIP,None,Y,D,NaN,NaN,Class B Common Units of Charter Communications...
569,CHTR,2023-12-07,2023-12-06,4,CHARTER COMMUNICATIONS IN,NEWHOUSE MICHAEL A,None,N,D,NaN,NaN,Class B Common Units of Charter Communications...
...,...,...,...,...,...,...,...,...,...,...,...,...
1518,CHTR,2010-09-14,NaT,RESTATED - 3,CHARTER COMMUNICATIONS IN,KARSH BRUCE A,None,N,None,NaN,NaN,Class A Common Stock
552,CHTR,2010-09-14,NaT,3,CHARTER COMMUNICATIONS IN,OAKTREE CAPITAL GROUP LLC,None,Y,None,NaN,NaN,Common Stock Warrants CUSIP 16117M123
553,CHTR,2010-09-14,NaT,3,CHARTER COMMUNICATIONS IN,OAKTREE CAPITAL GROUP LLC,None,Y,None,NaN,NaN,Class A Common Stock


## Screening for clustered Insider buying

Here we'll get data in the most recent 3 months and screen for insider buying

In [17]:
universe_path = './active_equities.csv'
universe_df = pd.read_csv(universe_path)

# Get three month ago date
today = datetime.today()
three_months_ago = today - timedelta(days=90)
formatted_date = three_months_ago.strftime('%Y-%m-%d')

transaction_code_map = {
    "P": "Purchase",
    "M": "Exercise of Derivative",
    "S": "Sale"
}

cluster_buying_companies = []

for index, row in universe_df.iterrows():

    insiders = nasdaqdatalink.get_table('SHARADAR/SF2', filingdate={'gte': formatted_date}, ticker=row['ticker'])
    insiders_df = pd.DataFrame(insiders, columns=["ticker", 'filingdate', 'transactiondate', 'formtype', 'issuername', 'ownername', 'officertitle', 'istenpercentowner', 'transactioncode', 'transactionpricepershare', 'transactionvalue', 'securitytitle'])
    insiders_df = insiders_df.sort_values(by="filingdate", ascending=False)

    # Check for any insider purchases in the last 3 months
    insider_buys_df = insiders_df[insiders_df['transactioncode'] == 'P']

    # Here we'll determine cluster insider buying to be 3 or more insiders
    if len(insider_buys_df) > 2:
        clustered_transaction_sum = insider_buys_df['transactionvalue'].sum()
        if clustered_transaction_sum > 1000000:
            row_values = {
                'ticker': row['ticker'],
                'companyName': row['name'],
                'numOfInsiders': len(insider_buys_df),
                'transactionTotal': clustered_transaction_sum
            }
            cluster_buying_companies.append(row_values)

clustered_buying_df = pd.DataFrame(cluster_buying_companies)
clustered_buying_df.to_csv('clusterBuying.csv', index=False)



KeyboardInterrupt: 